In [40]:
#### Dependencies ####
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [41]:
#### Loading the data ####
train_set = pd.read_csv("Data/train.csv")
test_set = pd.read_csv("Data/test.csv")

In [42]:
#### Getting rid of useless column ####
def drop_columns(data, columns):
    for column in columns:
        data.drop(column, axis=1, inplace=True)
    return data

train_set = drop_columns(train_set, ["PassengerId","Name", "Ticket", "Fare", "Cabin"])
test_set = drop_columns(test_set, ["PassengerId","Name", "Ticket", "Fare", "Cabin"])

In [43]:
#### Checking the data ####
def missing_report(dataset):
    for feature in dataset.columns:
        if np.dtype(dataset[feature]) == "object":
            # categorical feature
            misses = dataset[feature].isnull().sum() + dataset[feature].isna().sum() + dataset[feature].values.tolist().count("")
            if misses > 0:
                print(f"{feature} has {dataset[feature].isnull().sum()} missing values")
        else:
            # numerical feature
            misses = dataset[feature].isnull().sum() + dataset[feature].isna().sum()
            if misses > 0:
                print(f"{feature} has {dataset[feature].isnull().sum()} missing values")


print("Train report : ")
missing_report(train_set)

print("Test report : ")
missing_report(test_set)

# Age has missing values, we need to take care of it

Train report : 
Age has 177 missing values
Embarked has 2 missing values
Test report : 
Age has 86 missing values


In [44]:
#### Filling missing values ####
# For embarked, we will drop the missing values
train_set.dropna(subset=["Embarked"], inplace=True)

# For Age, we will perform a tracked imputation by filling the missing values with the median
def track_impute(dataset, feature):
    # index containing missing values
    missing_index = dataset[dataset[feature].isna()].index
    dataset[feature + 'Missing'] = False
    dataset.loc[missing_index, feature + 'Missing'] = True
    dataset[feature].fillna(dataset[feature].median(), inplace=True)
    return dataset

train_set = track_impute(train_set, "Age")
test_set = track_impute(test_set, "Age")

/tmp/ipykernel_15612/4094785868.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataset[feature].fillna(dataset[feature].median(), inplace=True)
/tmp/ipykernel_15612/4094785868.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, 

In [45]:
#### Performing AdaBoost ####
# Basic Classifier to compare with AdaBoost
print("Basic Classifier Accuracy : ",sum(train_set["Survived"] == 0)/len(train_set))

Basic Classifier Accuracy :  0.6175478065241845


In [46]:
#### Preprocessing the data ####
# We need to convert the categorical features to numerical, here label encoding is enough for 2 levels
def one_hot_encode(dataset, features):
    for feature in features:
        dataset = pd.concat([dataset, pd.get_dummies(dataset[feature], prefix=feature)], axis=1)
        dataset.drop(feature, axis=1, inplace=True)
    return dataset

# train_set = hot_one_encode(train_set,["Sex","AgeMissing"])
train_set = one_hot_encode(train_set,["Pclass","Sex","AgeMissing","Embarked"])
test_set = one_hot_encode(test_set,["Pclass","Sex","AgeMissing","Embarked"])

# We need to scale the numerical data to have a better performance
def scale_data(dataset, features):
    scaler = StandardScaler()
    dataset[features] = scaler.fit_transform(dataset[features])
    return dataset

train_set = scale_data(train_set, ["Age"])
test_set = scale_data(test_set, ["Age"])

In [47]:
train_set.head()

,Survived,Age,SibSp,Parch,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,AgeMissing_False,AgeMissing_True,Embarked_C,Embarked_Q,Embarked_S
0,0,-0.563674,1,0,False,False,True,False,True,True,False,False,False,True
1,1,0.669217,1,0,True,False,False,True,False,True,False,True,False,False
2,1,-0.255451,0,0,False,False,True,True,False,True,False,False,False,True
3,1,0.438050,1,0,True,False,False,True,False,True,False,False,False,True
4,0,0.438050,0,0,False,False,True,False,True,True,False,False,False,True


In [48]:
### Building the Log Reg model ###
X= train_set.drop("Survived", axis=1)
y = train_set["Survived"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
log_reg = LogisticRegression()
log_reg_model = log_reg.fit(X_train,y_train)

In [49]:
#### Predicting the test set ####
y_preds = log_reg_model.predict(X_test)
print("Accuracy : ", metrics.accuracy_score(y_test, y_preds))  # this is better than the basic classifier

Accuracy :  0.8146067415730337


In [50]:
#### Fit on the whole dataset #####
log_reg_model = log_reg.fit(X,y)

In [51]:
#### Predicting the test set and render it to the Kaggle submission format ####
titanic_preds = log_reg_model.predict(test_set)
submission = pd.DataFrame({"PassengerId": range(892,1310), "Survived": titanic_preds})
submission_path = "Data/titanic_submission.csv"

# Save the CSV file to the repository
with open(submission_path, "w") as file:
    submission.to_csv(file, index=False)

print("finished")

finished
